In [7]:
import cv2
import mediapipe as mp
import numpy as np
import os

In [8]:
# ตั้งค่า MediaPipe Holistic
mp_holistic = mp.solutions.holistic
mp_draw = mp.solutions.drawing_utils
holistic = mp_holistic.Holistic()

# สร้างโฟลเดอร์เก็บข้อมูล
DATA_DIR = "sign_data"
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

cap = cv2.VideoCapture(0)

label = input("Enter label for sign (e.g., 'hello', 'thanks'): ")
label_dir = os.path.join(DATA_DIR, label)
if not os.path.exists(label_dir):
    os.makedirs(label_dir)

count = 0
while count < 100:  # เก็บ 100 ตัวอย่าง
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = holistic.process(rgb_frame)

    if results.pose_landmarks and results.face_landmarks and results.left_hand_landmarks and results.right_hand_landmarks:
        mp_draw.draw_landmarks(frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
        mp_draw.draw_landmarks(frame, results.face_landmarks, mp.solutions.face_mesh.FACEMESH_TESSELATION)
        mp_draw.draw_landmarks(frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        mp_draw.draw_landmarks(frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

        # ดึงค่า Landmark ทั้งหมด
        landmarks = []
        for lm in results.pose_landmarks.landmark:  # ไหล่ แขน
            landmarks.append([lm.x, lm.y, lm.z])
        for lm in results.face_landmarks.landmark:  # ใบหน้า
            landmarks.append([lm.x, lm.y, lm.z])
        for lm in results.left_hand_landmarks.landmark:  # มือซ้าย
            landmarks.append([lm.x, lm.y, lm.z])
        for lm in results.right_hand_landmarks.landmark:  # มือขวา
            landmarks.append([lm.x, lm.y, lm.z])

        landmarks = np.array(landmarks).flatten()
        np.save(os.path.join(label_dir, f"{count}.npy"), landmarks)
        count += 1

    cv2.imshow("Sign Language Data Collection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Enter label for sign (e.g., 'hello', 'thanks'):  Bye


In [3]:
print("Original X shape:", X.shape)

NameError: name 'X' is not defined

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.layers import Reshape
import numpy as np
import os

# โหลดข้อมูล
DATA_DIR = "sign_data"
labels = os.listdir(DATA_DIR)
label_map = {label: idx for idx, label in enumerate(labels)}

X, y = [], []
for label in labels:
    files = os.listdir(os.path.join(DATA_DIR, label))
    for file in files:
        data = np.load(os.path.join(DATA_DIR, label, file))
        X.append(data)
        y.append(label_map[label])

X = np.array(X)
y = np.array(y)

# Reshape ข้อมูลให้เหมาะกับ LSTM
X = X.reshape(X.shape[0], X.shape[1], 1)
print("New X shape:", X.shape)

model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X.shape[1], 1)),  # (Timesteps, Features)
    Dropout(0.3),
    LSTM(32),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(len(labels), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# เทรนโมเดล
model.fit(X, y, epochs=20, validation_split=0.2)
model.save("sign_language_model.h5")



New X shape: (300, 1629, 1)
Epoch 1/20


C:\Users\kimza\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 578ms/step - accuracy: 0.4148 - loss: 1.0825 - val_accuracy: 0.0000e+00 - val_loss: 1.5004
Epoch 2/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 512ms/step - accuracy: 0.4219 - loss: 1.0389 - val_accuracy: 0.0000e+00 - val_loss: 1.9086
Epoch 3/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 502ms/step - accuracy: 0.4194 - loss: 1.0378 - val_accuracy: 0.0000e+00 - val_loss: 1.6465
Epoch 4/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 499ms/step - accuracy: 0.4634 - loss: 1.0215 - val_accuracy: 0.0000e+00 - val_loss: 1.6715
Epoch 5/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 512ms/step - accuracy: 0.4662 - loss: 1.0088 - val_accuracy: 0.0000e+00 - val_loss: 1.8144
Epoch 6/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 496ms/step - accuracy: 0.5610 - loss: 0.9568 - val_accuracy: 0.0000e+00 - val_loss: 1.8447
Epoch 7/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 514ms/step - accuracy: 0.5503 - loss: 0.9435 - val_accuracy: 0.0000e+00 - val_loss: 1.5954
Epoch 8/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 519ms/step - accuracy: 0.5148 - loss: 0.9151 - val_accuracy: 0

In [ ]:
# โหลดโมเดลที่เทรนไว้
model = tf.keras.models.load_model("sign_language_model.h5")

mp_holistic = mp.solutions.holistic
mp_draw = mp.solutions.drawing_utils
holistic = mp_holistic.Holistic()

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = holistic.process(rgb_frame)

    if results.pose_landmarks and results.face_landmarks and results.left_hand_landmarks and results.right_hand_landmarks:
        mp_draw.draw_landmarks(frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
        mp_draw.draw_landmarks(frame, results.face_landmarks, mp.solutions.face_mesh.FACEMESH_TESSELATION)
        mp_draw.draw_landmarks(frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        mp_draw.draw_landmarks(frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

        # ดึงค่า Landmark
        landmarks = []
        for lm in results.pose_landmarks.landmark:
            landmarks.append([lm.x, lm.y, lm.z])
        for lm in results.face_landmarks.landmark:
            landmarks.append([lm.x, lm.y, lm.z])
        for lm in results.left_hand_landmarks.landmark:
            landmarks.append([lm.x, lm.y, lm.z])
        for lm in results.right_hand_landmarks.landmark:
            landmarks.append([lm.x, lm.y, lm.z])

        landmarks = np.array(landmarks).flatten().reshape(1, -1)

        # ทำนายภาษามือ
        prediction = model.predict(landmarks)
        label = np.argmax(prediction)

        # แสดงผลบนหน้าจอ
        cv2.putText(frame, f"Prediction: {list(label_map.keys())[label]}", (10, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    cv2.imshow("Sign Language Recognition", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━

In [1]:
pip install PyQt5

Note: you may need to restart the kernel to use updated packages.


In [11]:
import sys
import os
import cv2
import numpy as np
import tensorflow as tf
import mediapipe as mp
from PyQt5.QtWidgets import QApplication, QLabel, QWidget, QVBoxLayout
from PyQt5.QtGui import QImage, QPixmap
from PyQt5.QtCore import QTimer

In [ ]:
# โหลดโมเดลที่เทรนไว้
model = tf.keras.models.load_model("sign_language_model.h5")
DATA_DIR = "sign_data"
labels = os.listdir(DATA_DIR)
label_map = {label: idx for idx, label in enumerate(labels)}

mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic()

class SignLanguageApp(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Sign Language Recognition with PyQt")
        self.setGeometry(100, 100, 800, 600)
        
        self.image_label = QLabel(self)
        layout = QVBoxLayout()
        layout.addWidget(self.image_label)
        self.setLayout(layout)

        self.cap = cv2.VideoCapture(0)
        self.timer = QTimer()
        self.timer.timeout.connect(self.update_frame)
        self.timer.start(30)  # 30 ms ต่อเฟรม

    def update_frame(self):
        ret, frame = self.cap.read()
        if not ret:
            return
        
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(rgb_frame)

        if results.pose_landmarks and results.face_landmarks and results.left_hand_landmarks and results.right_hand_landmarks:
            # ดึงค่า Landmark
            landmarks = []
            for lm in results.pose_landmarks.landmark:
                landmarks.append([lm.x, lm.y, lm.z])
            for lm in results.face_landmarks.landmark:
                landmarks.append([lm.x, lm.y, lm.z])
            for lm in results.left_hand_landmarks.landmark:
                landmarks.append([lm.x, lm.y, lm.z])
            for lm in results.right_hand_landmarks.landmark:
                landmarks.append([lm.x, lm.y, lm.z])

            landmarks = np.array(landmarks).flatten().reshape(1, -1)

            # ทำนายผล
            prediction = model.predict(landmarks)
            label = np.argmax(prediction)
            text = f"Prediction: {list(label_map.keys())[label]}"
            
            cv2.putText(frame, text, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # แปลงภาพจาก OpenCV ไปเป็น QImage
        height, width, channel = frame.shape
        bytes_per_line = channel * width
        q_img = QImage(frame.data, width, height, bytes_per_line, QImage.Format_RGB888)
        pixmap = QPixmap.fromImage(q_img)
        self.image_label.setPixmap(pixmap)

    def closeEvent(self, event):
        self.cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = SignLanguageApp()
    window.show()
    sys.exit(app.exec_())

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━